In [1]:
import ray
import ray.rllib

import q1physrl.env

/home/matt/.virtualenvs/100m-rl/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/home/matt/.virtualenvs/100m-rl/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
ray.init()

2020-01-26 18:30:58,183	WARNING worker.py:673 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.
2020-01-26 18:30:58,196	WARNING services.py:597 -- setpgrp failed, processes may not be cleaned up properly: [Errno 1] Operation not permitted.
2020-01-26 18:30:58,230	INFO resource_spec.py:216 -- Starting Ray with 0.98 GiB memory available for workers and up to 0.5 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-01-26 18:30:58,547	WARNING services.py:1009 -- Failed to start the reporter. The reporter requires 'pip install psutil'.


{'node_ip_address': '192.168.178.24',
 'redis_address': '192.168.178.24:21823',
 'object_store_address': '/tmp/ray/session_2020-01-26_18-30-58_194544_16379/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-01-26_18-30-58_194544_16379/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2020-01-26_18-30-58_194544_16379'}

In [3]:
trainer = ray.rllib.agents.a3c.A2CTrainer(
    env=q1physrl.env.PhysEnv,
    config={"env_config": {"num_envs": 100}, "gamma": 0.9}
)

2020-01-26 18:30:58,792	INFO trainer.py:371 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
2020-01-26 18:30:58,833	INFO trainer.py:512 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2020-01-26 18:31:00,607	WARNING util.py:45 -- Install gputil for GPU system monitoring.
2020-01-26 18:31:00,608	WARNING util.py:48 -- Install psutil to monitor system performance.


In [4]:
while True:
    trainer.train()['episode_reward_mean']

(pid=16422) /home/matt/.virtualenvs/100m-rl/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
(pid=16422)   warnings.warn(msg)
(pid=16423) /home/matt/.virtualenvs/100m-rl/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
(pid=16423)   warnings.warn(msg)
(pid=16422) /home/matt/.virtualenvs/100m-rl/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
(pid=16422)   warnings.warn(msg)
(pid=16423) /home/matt/.virtualenvs/100m-rl/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma modu

/home/matt/.virtualenvs/100m-rl/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/matt/.virtualenvs/100m-rl/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


KeyboardInterrupt: 

In [6]:
env = q1physrl.env.PhysEnv({'num_envs': 1})
obs = env.vector_reset()
done = None

while done is None or not done[0]:
    action = trainer.compute_action(obs[0])
    print(action)
    obs, reward, done, _ = env.vector_step(action[None])
    print(obs)

[2 1]
[[ 90.     32.375  16.     16.    -16.   ]]
[2 1]
[[ 90.  32.  16.  16. -32.]]
[0 1]
[[ 90.     31.375   0.     32.    -32.   ]]
[2 1]
[[ 90.   30.5   0.   32.  -48. ]]
[2 1]
[[ 90.     29.625   0.     32.    -64.   ]]
[2 1]
[[ 90.   28.5   0.   32.  -64. ]]
[2 1]
[[ 90.    27.25   0.    32.   -80.  ]]
[2 1]
[[ 90.    25.75   0.    32.   -96.  ]]
[2 1]
[[  90.     24.25    0.     32.   -112.  ]]
[0 1]
[[90. 24.  0. 32.  0.]]
[2 1]
[[90. 24. 16. 64.  0.]]
[2 1]
[[90. 24. 48. 96.  0.]]
[0 1]
[[ 90.  24.  16. 112.   0.]]
[0 1]
[[ 90.  24.   0. 144.   0.]]
[2 1]
[[ 90.  24.  16. 176.   0.]]
[2 1]
[[ 90.  24.  48. 192.   0.]]
[2 1]
[[ 90.  24.  64. 224.   0.]]
[2 1]
[[ 90.  24.  96. 240.   0.]]
[0 1]
[[ 90.  24.  64. 256.   0.]]
[2 1]
[[ 90.  24.  80. 272.   0.]]
[2 1]
[[ 90.  24. 112. 288.   0.]]
[0 1]
[[ 90.  24.  80. 304.   0.]]
[2 1]
[[ 90.  24.  96. 320.   0.]]
[0 1]
[[ 90.  24.  64. 336.   0.]]
[0 1]
[[ 90.  24.  32. 352.   0.]]
[0 1]
[[ 90.  24.   0. 368.   0.]]
[0 1]
[[ 90.  2

[0 1]
[[ 90.  24.   0. 416.   0.]]
[2 1]
[[ 90.  24.   0. 416.   0.]]
[2 1]
[[ 90.  24.  16. 400.   0.]]
[0 1]
[[ 90.  24.   0. 416.   0.]]
[2 1]
[[ 90.  24.   0. 400.   0.]]
[0 1]
[[ 90.  24.   0. 416.   0.]]
[2 1]
[[ 90.  24.   0. 416.   0.]]
[2 1]
[[ 90.  24.  16. 400.   0.]]
[0 1]
[[ 90.  24.   0. 416.   0.]]
[2 1]
[[ 90.  24.   0. 400.   0.]]
[0 1]
[[ 90.  24.   0. 416.   0.]]
[2 1]
[[ 90.  24.   0. 416.   0.]]
[2 1]
[[ 90.  24.  16. 400.   0.]]
[0 1]
[[ 90.  24.   0. 416.   0.]]
[2 1]
[[ 90.  24.   0. 400.   0.]]
[0 1]
[[ 90.  24.   0. 416.   0.]]
[2 1]
[[ 90.  24.   0. 416.   0.]]
[2 1]
[[ 90.  24.  16. 400.   0.]]
[0 1]
[[ 90.  24.   0. 416.   0.]]
[2 1]
[[ 90.  24.   0. 400.   0.]]
[0 1]
[[ 90.  24.   0. 416.   0.]]
[2 1]
[[ 90.  24.   0. 416.   0.]]
[2 1]
[[ 90.  24.  16. 400.   0.]]
[0 1]
[[ 90.  24.   0. 416.   0.]]
[2 1]
[[ 90.  24.   0. 400.   0.]]
[0 1]
[[ 90.  24.   0. 416.   0.]]
[2 1]
[[ 90.  24.   0. 416.   0.]]
[2 1]
[[ 90.  24.  16. 400.   0.]]
[0 1]
[[ 90.  24.   